# ItemItem Result Analysis

In [1]:
import pandas as pd
import numpy as np

## Compare Predictions

In [2]:
java = pd.read_csv('build/lenskit/item-item-ml-100k-preds.csv')
java.head()

,user,item,prediction
0,770,1050,4.627339
1,770,98,5.284142
2,770,1066,3.755587
3,770,1260,3.733646
4,770,1499,3.822878


In [4]:
lkpy = pd.read_csv('build/lkpy/item-item-ml-100k-preds.csv')
lkpy.head()

,user,item,prediction
0,2,967,3.244902
1,2,1671,NaN
2,2,1467,NaN
3,2,1544,NaN
4,2,649,NaN


In [5]:
merged = pd.merge(java.rename(columns={'prediction': 'java'}), lkpy.rename(columns={'prediction': 'lkpy'}))
merged.head()

,user,item,java,lkpy
0,770,1050,4.627339,4.627339
1,770,98,5.284142,NaN
2,770,1066,3.755587,3.755587
3,770,1260,3.733646,3.733646
4,770,1499,3.822878,3.822878


In [6]:
merged['diff'] = merged.java - merged.lkpy
merged['diff'].describe()

count    4.940000e+02
mean    -2.842731e-02
std      1.283738e-01
min     -1.115043e+00
25%     -4.440892e-16
50%      0.000000e+00
75%      0.000000e+00
max      8.201200e-01
Name: diff, dtype: float64